<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat demo

(not so risky)

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
!pip install git+https://github.com/graphistry/pygraphistry.git@cuCat
!pip install -U git+https://github.com/graphistry/cu-cat.git@cu-cat #@0.01.0

In [ ]:
import cuml
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [ ]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /usr/local/lib/python3.8/dist-packages/cu_cat/

In [ ]:
from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.8/dist-packages/cu_cat/__init__.py


In [ ]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [ ]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__


'0.28.6+17.g21dac04'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install dirty_cat
import dirty_cat

In [ ]:
import logging
logger = logging.getLogger()


## ask HN

In [ ]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = df.sample(10000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


# g1a = g.featurize(feature_engine='cuCat',memoize=False)
g1aa = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cucat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         24507879 function calls (24451692 primitive calls) in 105.511 seconds

   Ordered by: internal time
   List reduced from 2314 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       13   35.673    2.744   51.472    3.959 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:987(_multiplicative_update_h_smallfast)
      242    9.948    0.041   17.904    0.074 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:935(_multiplicative_update_h)
        9    9.291    1.032    9.292    1.032 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       30    7.924    0.264    7.924    0.264 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
       21    4.783    0.228   11.469    0.546 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1184(_count_vocab)
      123    3.663    0.030    3.663    0.030 {built-in method scipy.sparse._sparsetools.csc_tocsr}
     945

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


# g1b = g.featurize(feature_engine='dirty_cat',memoize=False)
g1bb = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         35574969 function calls (35518798 primitive calls) in 152.723 seconds

   Ordered by: internal time
   List reduced from 2111 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      764   41.615    0.054   87.567    0.115 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
4218044/4217259   40.498    0.000   50.343    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      360   13.848    0.038   33.832    0.094 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
        9    8.602    0.956    8.603    0.956 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       21    5.399    0.257   12.731    0.606 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1184(_count_vocab)
  3821176    4.685    0.000   44.981    0.000 <__array_function__ internals>:177(dot)
    15120 

## redteam

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g2a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         12139718 function calls (12099306 primitive calls) in 68.897 seconds

   Ordered by: internal time
   List reduced from 1889 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5   18.967    3.793   21.693    4.339 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:987(_multiplicative_update_h_smallfast)
        3   17.173    5.724   17.173    5.724 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    7.701    0.000    9.616    0.000 /usr/local/lib/python3.8/dist-packages/pandas/core/missing.py:60(mask_missing)
42595/42360    7.611    0.000   24.918    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
    79468    1.920    0.000    1.920    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     4244    1.872    0.000    1.932    0.000 /usr/local/lib/python3.8/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
        7    1.180    0.169    3.414    0

In [ ]:
%%time
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g2b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         30354021 function calls (30313610 primitive calls) in 100.076 seconds

   Ordered by: internal time
   List reduced from 1783 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
2899279/2899038   18.149    0.000   35.065    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3   16.697    5.566   16.697    5.566 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     1054   14.508    0.014   44.162    0.042 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
    39524    7.975    0.000   10.027    0.000 /usr/local/lib/python3.8/dist-packages/pandas/core/missing.py:60(mask_missing)
   134889    3.527    0.000   10.602    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/compressed.py:136(check_format)
  2310751    3.488    0.000   13.876    0.000 <__array_function__ internals>:177(dot)
      775    3.329    0.004    9.84

## CTU-13 malware dataset

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g3a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         4639229 function calls (4496707 primitive calls) in 27.471 seconds

   Ordered by: internal time
   List reduced from 2074 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    5.216    0.580    5.216    0.580 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        3    1.801    0.600    1.801    0.600 /usr/local/lib/python3.8/dist-packages/numpy/ma/core.py:3211(__getitem__)
      138    1.636    0.012    1.636    0.012 {method 'argsort' of 'numpy.ndarray' objects}
       15    1.583    0.106    2.070    0.138 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:987(_multiplicative_update_h_smallfast)
    13182    1.163    0.000    3.666    0.000 /usr/local/lib/python3.8/dist-packages/scipy/stats/_distn_infrastructure.py:2096(ppf)
251588/193809    1.092    0.000   12.425    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      319    0.973    0.003    

In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g3b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         15716281 function calls (15573737 primitive calls) in 68.222 seconds

   Ordered by: internal time
   List reduced from 1950 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      137    9.182    0.067    9.182    0.067 {method 'argsort' of 'numpy.ndarray' objects}
     1389    6.867    0.005   23.653    0.017 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
1699910/1642103    5.898    0.000   20.008    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3    5.209    1.736    5.209    1.736 /usr/local/lib/python3.8/dist-packages/numpy/ma/core.py:3211(__getitem__)
        9    4.484    0.498    4.484    0.498 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    13375    3.468    0.000    3.468    0.000 {method 'take' of 'numpy.ndarray' objects}
    90921    2.257    0.000    6.908    0.000 /usr/local/lib/python3.8/dist-

## 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 1000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g4a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g4b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

## winlogs

### this example finds the middle ground (1e9 > samples * features > 2e9) where cpu is faster

In [ ]:
winlogs=pd.read_parquet('../part.88 (1).parquet')
winlogs=winlogs.sample(50000,replace=False).iloc[:,:10]

In [ ]:
# import cupy as cp
# del g,g2,g3
# cp._default_memory_pool.free_all_blocks()

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g5a = g.featurize(feature_engine='cu_cat',memoize=False)
# g5aa = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cucat_20k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g5b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g5bb = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_dirtycat_20k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
### stopppp

### this large dataset uses same dirty_cat loop just passing to gpu

*   smaller examples process everything at once on gpu for speedup



In [ ]:
winlogs=pd.read_parquet('../part.88 (1).parquet') #### run on EKS (>time limit on colab)
winlogs=winlogs.sample(100000,replace=False).iloc[:,:10]

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g2 = g.featurize(feature_engine='cu_cat',memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cucat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         126150727 function calls (124547064 primitive calls) in 1731.649 seconds

   Ordered by: internal time
   List reduced from 1590 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1328  728.858    0.549 1282.678    0.966 /notebooks/cuCat/cuCat/_gap_encoder.py:935(_multiplicative_update_h)
       21  348.883   16.613  348.902   16.614 /opt/conda/envs/rapids/lib/python3.8/site-packages/numpy/lib/arraysetops.py:519(in1d)
  7424650  293.035    0.000  293.311    0.000 {method 'dot' of 'cupy._core.core.ndarray' objects}
13700193/13698616   81.387    0.000  739.918    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   170995   25.538    0.000   30.131    0.000 /opt/conda/envs/rapids/lib/python3.8/site-packages/cupyx/scipy/sparse/_index.py:43(_get_csr_submatrix_major_axis)
     1888   25.283    0.013   25.322    0.013 /opt/conda/envs/rapids/lib/python3.8/site-packages/cupyx/scipy/sparse/compressed.p

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g2a = g.featurize(feature_engine='dirty_cat')#,memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_dirtycat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         229334878 function calls (227731084 primitive calls) in 4362.954 seconds

   Ordered by: internal time
   List reduced from 1382 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
26444345/26442694 2848.058    0.000 3196.088    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    27370  482.535    0.018 3428.776    0.125 /notebooks/dirty_cat/dirty_cat/_gap_encoder.py:953(_multiplicative_update_w)
       21  338.738   16.130  338.757   16.131 /opt/conda/envs/rapids/lib/python3.8/site-packages/numpy/lib/arraysetops.py:519(in1d)
    29380  157.298    0.005  488.620    0.017 /notebooks/dirty_cat/dirty_cat/_gap_encoder.py:985(_multiplicative_update_h)
    27370   62.879    0.002   80.106    0.003 /notebooks/dirty_cat/dirty_cat/_gap_encoder.py:943(_rescale_W)
 20130688   37.406    0.000 2881.503    0.000 <__array_function__ internals>:2(dot)
  1492801   36.721    0.000  126.308    0.000 /opt/conda/envs/